In [9]:
import sys

# pandas读写csv文件
import pandas as pd
import numpy as np

path = '/Users/mingjue/downloads/'

df_train_1 = pd.read_csv(path + 'data_format1/train_format1.csv')
df_test_1 = pd.read_csv(path + 'data_format1/test_format1.csv')
df_user_info = pd.read_csv(path + 'data_format1/user_info_format1.csv')
df_user_log = pd.read_csv(path + 'data_format1/user_log_format1.csv')

df_train_2 = pd.read_csv(path + 'data_format2/train_format2.csv')
df_test_2 = pd.read_csv(path + 'data_format2/test_format2.csv')

df_infer = pd.read_csv(path + 'sample_submission.csv')


In [12]:
df_user_log

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0
...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1110,0
54925326,208016,1058313,898,1346,7995.0,1110,0
54925327,208016,449814,898,983,7995.0,1110,0
54925328,208016,634856,898,1346,7995.0,1110,0


In [8]:
df_test


,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN
...,...,...,...
261472,228479,3111,NaN
261473,97919,2341,NaN
261474,97919,3971,NaN
261475,32639,3536,NaN


In [6]:
df_infer

,user_id,merchant_id,prob
0,163968,4605,0.5
1,360576,1581,0.5
2,98688,1964,0.5
3,98688,3645,0.5
4,295296,3361,0.5
...,...,...,...
261472,228479,3111,0.5
261473,97919,2341,0.5
261474,97919,3971,0.5
261475,32639,3536,0.5
